In [6]:
from os import listdir
import os
import json
import pandas as pd
from matplotlib.pyplot import xlabel
from typing import List, Dict

import pandas as pd

In [7]:
def load_solutions(output_dir: str) -> List[Dict]:
    file_names = os.listdir(output_dir)
    bh_files = [f for f in file_names if f.startswith("BH_")]
    urjc_files = [f for f in file_names if f.startswith("URJC_")]
    upv_delay_files = [f for f in file_names if f.startswith("UPVD_")]
    ws_delay_files = [f for f in file_names if f.startswith("WSD_")]
    #urjc_files = [f for f in file_names if f not in upv_files+bh_files+ws_files and not f.startswith("WS_iterations_")]
    

    urjc_solutions = load_json_solutions([os.path.join(output_dir, f) for f in urjc_files])
    bh_solutions = load_json_solutions([os.path.join(output_dir, f) for f in bh_files])
    upv_delay_solutions = load_json_solutions([os.path.join(output_dir, f) for f in upv_delay_files])
    ws_delay_solutions = load_json_solutions([os.path.join(output_dir, f) for f in ws_delay_files])

    return urjc_solutions, bh_solutions, upv_delay_solutions, ws_delay_solutions

def load_json_solutions(file_paths: List[str]) -> List[Dict]:
    return [json.load(open(path)) for path in file_paths]

def get_best_solution(solutions: List[Dict]) -> Dict:
    return min(solutions, key=lambda x: x['computational_time'])
    #return min(solutions, key=lambda x: x['cost'])

def extract_number_by_string(input_string: List[str], pattern : str) -> List[int]:
    rqt_numbers = []
    groups_letters = input_string.split("_")
    if not pattern:
        return groups_letters[-1]
    
    for group in groups_letters:
        if pattern in group:
            end = group.find(pattern)
            return int(group[:end])

    return "Not Found"

def extract_results(results_json, exp_graph, f, algorithm):
    
        used_vehicles_costs = [cost for cost in results_json["costs_routes"].values() if cost > 0]
        
        return {
            "graph" : exp_graph,
            "instance" : f,
            "algorithm": algorithm,
            "requests": extract_number_by_string(results_json["instance_name"], "RQT"),
            "robots" : extract_number_by_string(results_json["instance_name"], "HR"),
            "capacity" : extract_number_by_string(results_json["instance_name"], "Q"),
            "demand" : extract_number_by_string(results_json["instance_name"], "q"),
            "num_instance" : extract_number_by_string(results_json["instance_name"], None),
            "isBigTW": bool("bigTW" in results_json["instance_name"]),
            "cost": results_json['cost'],
            "time": results_json['max_time'],
            "distance" : results_json['distance_travelled'],
            "used_vehicles": len(used_vehicles_costs),
            "used_max": max(used_vehicles_costs),
            "used_min": min(used_vehicles_costs),
            "used_mean": sum(used_vehicles_costs) / len(used_vehicles_costs),
            "computational_time" : results_json['computational_time'],
            "delay" : results_json.get('delay', 0),
            "delay_cost" : results_json.get('delay_cost', 0)
        }

def add_empty_results(exp_graph, f, algorithm):

        return {
            "graph" : exp_graph,
            "instance" : f,
            "algorithm": algorithm,
            "requests": extract_number_by_string(f, "RQT"),
            "robots" : extract_number_by_string(f, "HR"),
            "capacity" : extract_number_by_string(f, "Q"),
            "demand" : extract_number_by_string(f, "q"),
            "num_instance" : extract_number_by_string(f, None),
            "isBigTW": bool("bigTW" in f),
            "cost": None,
            "time": None,
            "distance" : None,
            "used_vehicles": None,
            "used_max": None,
            "used_min": None,
            "used_mean": None,
            "computational_time" : None,
            "delay" : None,
            "delay_cost" : None
        }
        
def add_results_or_empty(all_results: list, solutions, exp_graph, experiment, algorithm):
    try:
        best_f = get_best_solution(solutions)
        if algorithm in ["WSHeuristic", "Heuristic"] and best_f["invalid"]:
            raise ValueError("The solution is invalid.")
        results = extract_results(best_f, exp_graph, experiment, algorithm)
        all_results.append(results)
        return True
    except (KeyError, ValueError):
        results = add_empty_results(exp_graph, experiment, algorithm)
        all_results.append(results)
        return False


In [9]:
results_name_file = "results.csv"
experiments_graph = ["random", "mountain", "asymmetric_grid", "symmetric_grid"] 


all_results = []
fail_experiments = []
for exp_graph in experiments_graph:
    path = f"{exp_graph}" # Path to the folder containing the experiments
    exp_files = listdir(path)
    for experiment in exp_files:
        urjc_solutions, bh_solutions, upv_delay_solutions, ws_delay_solutions = load_solutions(f"{path}/{experiment}/output")

        if not add_results_or_empty(all_results, urjc_solutions, exp_graph, experiment, "MILP"):
            print(f"Error: No MILP solutions found for {path}/{experiment}")
            fail_experiments.append(f"{path}/MILP/{experiment}")
            
        
        if not add_results_or_empty(all_results, bh_solutions, exp_graph, experiment, "SCFI"):
            print(f"Error: No Base Heuristic solutions found for {path}/{experiment}")
            fail_experiments.append(f"{path}/SCFI/{experiment}")
            
    
        
        if not add_results_or_empty(all_results, upv_delay_solutions, exp_graph, experiment, "MCFI"):
            print(f"Error: No UPV Delay solutions found for {path}/{experiment}")
            fail_experiments.append(f"{path}/MCFI/{experiment}")

        if not add_results_or_empty(all_results, ws_delay_solutions, exp_graph, experiment, "WS-MILP"):
            print(f"Error: No WS Delay solutions found for {path}/{experiment}")
            fail_experiments.append(f"{path}/WS-MILP/{experiment}")

results_df = pd.DataFrame(all_results)
results_df.to_csv(results_name_file, index=False)

Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_8
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_7
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_3
Error: No MILP solutions found for asymmetric_grid/6HR_6Q_50RQT_2q_bigTW_8
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_6
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_2
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_0
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_1
Error: No MILP solutions found for asymmetric_grid/6HR_6Q_50RQT_2q_bigTW_2
Error: No MILP solutions found for asymmetric_grid/6HR_6Q_50RQT_2q_bigTW_5
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_4
Error: No MILP solutions found for asymmetric_grid/9HR_6Q_50RQT_2q_bigTW_9
Error: No MILP solutions found for asymmetric_grid/6HR_6Q_50RQT_2q_bigTW_3
Error: No MILP solutions 